In [71]:
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from random import sample
from sys import stdout
import pandas as pd

In [14]:
# INSTANCE BROWSER
def openBrowser(url, exchange=None):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    import platform
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
#         profile_path = '/home/nomadbitcoin/whatsapp_profiles/' + business 
    elif platform.system() == 'Windows':
        driver_path = r'C:\\Users\\Yan\\Desktop\\brisbane_bot\\webdriver\\win\\chromedriver.exe'
        profile_path = "C:\\Users\\Yan\\Desktop\\trading_betfair\\browser_profile\\" + exchange
        print(profile_path)
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
        driver.get(url)
        return True
    except Exception as error:
        print(type(error), error)
        return False

In [15]:
def openGames():

    championship_list = driver.find_elements_by_class_name('ufm-MarketGroupButtonUpcomingCompetition')

    for count, championship in enumerate(championship_list):
        try:
            championship.click()
            time.sleep(0.3)
        except Exception as error:        
#             print(type(error), error)
            pass
            
    return True

In [105]:
games_next_3_hrs = 'https://www.bet365.com/#/AC/B1/C1/D13/E0/F2/J0/Q1/F^3'
games_next_12_hrs = 'https://www.bet365.com/#/AC/B1/C1/D13/E0/F2/J0/Q1/F^12'
games_next_24_hrs = 'https://www.bet365.com/#/AC/B1/C1/D13/E0/F2/J0/Q1/F^24'

In [34]:
def getGames(just_randomize=False):
    if just_randomize == False:
        driver.get(games_next_3_hrs)
        time.sleep(1)
        openGames()    

        global htmlPage 
        htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
        campeonatos = htmlPage.find_all('div',{'class':'gll-MarketGroupContainer'})[1:] #primeiro item da lista vem vazio

        # LISTA PARA CADA TIPO DE DADO PARA PODER CRIAR RANK DE FORMA MAIS FACIL A SE LOCALIZAR NOS DICIONARIOS
        global championships, game_list, back_odds, home_odds, draw_odds, game_time
        championships = []
        game_list = []
        back_odds = []
        home_odds = []
        draw_odds = []
        game_time = []

        global data_games_today
        data_games_today = []

        for c in campeonatos:
            ''' pega todos os jogos que ha naquele campeonato '''
            games = c.find_all('div',{'class':'sl-CouponParticipantWithBookCloses'})
            for num, g in enumerate(games):
                try:
                    ''' PARA SALVAR OS DADOS SOBRE CADA PARTIDA '''
                    game_info = {'championship': '', 'time': '', 'game_name': '', 'back_team': '', 'home_team': '',
                                'back_odd': '', 'home_odd': '', 'draw_odd': ''}

                    ''' pega o nome do campeonato e salva na lista e tambem no dicionario de informacoes para cada partida'''
                    championship_name = c.find('div',{'class':'gll-MarketColumnHeader'}).text
                    championships.append(championship_name)
                    game_info['championship'] = championship_name 


                    ''' salva o nome do jogo no dicionario principal e tambem na lista de jogos'''
                    g_name = g.find('div',{'class':'sl-CouponParticipantWithBookCloses_Name'}).text
                    if ' v ' in g_name:
                        game_list.append(g_name)
                        game_info['game_name'] = g_name


                    ''' ira pegar os mercados -- tudo estar organizado em colunas entao os dados estao todos agrupados'''
                    markets = c.find_all('div',{'class':'ufm-MarketC4OddsSwitch'})
                    home, draw, back = markets[0], markets[1], markets[2]

                    game_info['home_odd'] = home.find_all('span',{'class':'gll-ParticipantOddsOnly_Odds'})[num].text #ira pegar o mercado home na posicao que esta o jogo
                    home_odds.append(game_info['home_odd'])
                    game_info['draw_odd'] = draw.find_all('span',{'class':'gll-ParticipantOddsOnly_Odds'})[num].text #ira pegar o mercado draw na posicao que esta o jogo
                    draw_odds.append(game_info['draw_odd'])
                    game_info['back_odd'] = back.find_all('span',{'class':'gll-ParticipantOddsOnly_Odds'})[num].text #ira pegar o mercado back na posicao que esta o jogo
                    back_odds.append(game_info['back_odd'])


                    ''' time do jogo'''
                    if g.find('div',{'class':'sl-CouponParticipantWithBookCloses_BookCloses'}) == None:
                        game_info['time'] = 'running'
                        game_time.append(game_info['time'])
                    else:
                        game_info['time'] = g.find('div',{'class':'sl-CouponParticipantWithBookCloses_BookCloses'}).text
                        game_time.append(game_info['time'])

                    ''' nome dos clubes'''
                    game_info['back_team'] = g_name[g_name.find(' v ') +3:]
                    game_info['home_team'] = g_name[:g_name.find(' v ')]

                    data_games_today.append(game_info)
                except Exception as error:        
                    print(type(error), error)
                    pass

    global equilibred_odds
    equilibred_odds = []

    global equilibred_game_name
    equilibred_game_name = []

    for game in data_games_today:
        if game['time'] != 'running' and 'Seg 11:' in game['time']:
            equit = float(game['home_odd']) - float(game['back_odd'])
            if str(equit).startswith('-'):
                equit = float(str(equit)[1:])
                mean = 1.5
                if equit <= mean:
                    equilibred_odds.append(equit)
                    equilibred_game_name.append(game['game_name'])
        #             print(equit)
        #             print('\n')

    
    
    # PEGA 5 JOGOS ALEATORIOS DA LISTA DE JOGOS EQUILIBRADOS COM MENOS DE 1 DE DIFERENCA DE ODDS
    for i in range(39):
        to_play = sample(equilibred_game_name, 1)
#         stdout.write('\r' + str(to_play))
        time.sleep(0.3)
    print('\n\nequit {} --> ESCOLHIDOS: \n'.format(mean))
    prinnt = [print('{} : {}'.format(num+1, game))for num, game in enumerate(to_play)]

In [18]:
openBrowser(games_next_3_hrs, 'bet365')

C:\Users\Yan\Desktop\trading_betfair\browser_profile\bet365


True

In [24]:
# getGames(just_randomize=False)
# equilibred_game_name

In [107]:
def minuteEvent():
    '''
        Ira pegar todos os jogos e entao pesquisar quais deles tem o mercado de "Minutos"
        
        Retorna duas listas, uma com todos os jogos e outra somente com os que possuem o mercado
    '''
    
    list_events = []
    for pos, game in enumerate(game_list):
        try:
            search_icon = driver.find_element_by_class_name('hm-MainHeaderRHSLoggedOutWide_Search')
            search_icon.click()
        except:
            print('search box not found')

        info = {'game_name': game, 'game_code': '', 'minute_market': False}

        time.sleep(1)
        search_box = driver.find_element_by_class_name('sml-SearchTextInput')

        '''  ira apagar os caracteres do jogo anterior'''
        for i in range(60):
            search_box.send_keys(Keys.BACKSPACE)

        time.sleep(1)
        search_box.send_keys(game)

        time.sleep(3)
        try:
            event_button = driver.find_element_by_class_name('ssm-SiteSearchLabelOnlyParticipant_Name')
            event_button.click()
        except Exception as error:
            if 'NoSuchElementException' in str(type(error)):
                print('\nNao encontrou o jogo {}\n'.format(game))    

        time.sleep(3)
        url_event = driver.current_url
        game_code = url_event.split('/')[-3]

        info['game_code'] = game_code if game_code.startswith('E') else None

        ''' encontra o mercado de ~Minutos~ '''
        htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')
        markets = htmlPage.find('div',{'class':'scr-ScrollableHorizontalNavBar_ButtonContainer'})
        if markets != None:
            for m in markets:
                if 'Minutos' in m.text:
                    info['minute_market'] = True

        list_events.append(info)

    games_with_minutemarket = []
    for event in list_events:
        if event['minute_market']:
            games_with_minutemarket.append(event)
    
    return games_with_minutemarket, list_events

In [106]:
driver.get(games_next_12_hrs)

In [108]:
openGames()

True

In [ ]:
a, b = minuteEvent()

In [112]:
# htmlPage = bs4.BeautifulSoup(driver.page_source, features='html.parser')

121

In [128]:
events_info = []
for event in htmlPage.find_all('div',{'class':'sl-CouponParticipantWithBookCloses'}):
    event_info = {'event_name': '', 'event_time': ''}
    
    event_name = event.find('div',{'class':'sl-CouponParticipantWithBookCloses_BookCloses'})
    if event_name != None:
        event_info['event_name'] = event_name.text
    
    event_time = event.find('div',{'class':'sl-CouponParticipantWithBookCloses_Name'})
    if event_time != None:
        event_info['event_time'] = event_time.text
    
    events_info.append(event_info)
#     break

In [130]:
events_info

AttributeError: 'list' object has no attribute 'values'

In [36]:
df = pd.DataFrame(data_games_today)
df

,back_odd,back_team,championship,draw_odd,game_name,home_odd,home_team,time,difference,goal_10minutes
30,2.70,Doncaster,Inglaterra - League One,3.20,Milton Keynes Dons v Doncaster,2.70,Milton Keynes Dons,Sab 12:00,0.000,None
379,2.37,CSC Sanmartin,Romênia - Liga III,3.60,CS Luceafarul Oradea v CSC Sanmartin,2.37,CS Luceafarul Oradea,Sab 10:00,0.000,None
265,2.60,K Sibenik,Croácia - 2.HNL,3.10,HNK Sibenik,2.60,HNK Sibeni,Sab 09:00,0.000,None
179,2.60,TSV Landsberg,Alemanha - Oberliga - Bayern South,3.20,SV Pullach v TSV Landsberg,2.60,SV Pullach,Sab 10:00,0.000,None
278,2.87,nløse,Dinamarca - 2.ª Divisão - Grupo 1,2.75,Vanløse,2.87,Vanløs,Sab 09:00,0.000,None
545,2.45,Escócia - Feminino,Feminino - Internacional,3.50,Islândia - Feminino v Escócia - Feminino,2.45,Islândia - Feminino,Sab 11:00,0.000,None
461,3.10,ni Altindag Belediyespor,Turquia - 3 Lig - Grupo 3,2.25,Yeni Altindag Belediyespor,3.10,Yeni Altindag Belediyespo,running,0.000,None
320,2.62,zito FC,Quênia - Premier League,3.10,Wazito FC,2.60,Wazito F,Sab 09:00,0.020,None
87,2.62,Dundee,Escócia - Championship,3.30,Ayr v Dundee,2.60,Ayr,Sab 12:00,0.020,None
408,2.55,FK Poprad,Eslováquia - 2. Liga,3.40,FC Kosice v FK Poprad,2.45,FC Kosice,Sab 10:30,0.100,None


In [22]:
# Criando nova coluna para diferenca de odds
df['difference'] = 0.01

#coluna para inserir o resultado se teve gol antes dos 10 minutos ou nao
df['goal_10minutes'] = None

In [23]:
# INSERINDO O VALOR DE DIFERENCA ENTRE O TIME DA CASA E TIME DE FORA
for index, row in df.iterrows():
    difference = float(float(df.iloc[index]['home_odd']) - float(df.iloc[index]['back_odd']))
    
    if str(difference).startswith('-'):
        difference = float(str(difference)[1:])
    
    df.loc[index,'difference'] =  difference 

In [24]:
# REORDENA PELA MENOR DIFERENCA
df = df.sort_values(by=['difference'])

In [25]:
df

,back_odd,back_team,championship,draw_odd,game_name,home_odd,home_team,time,difference,goal_10minutes
30,2.70,Doncaster,Inglaterra - League One,3.20,Milton Keynes Dons v Doncaster,2.70,Milton Keynes Dons,Sab 12:00,0.000,None
379,2.37,CSC Sanmartin,Romênia - Liga III,3.60,CS Luceafarul Oradea v CSC Sanmartin,2.37,CS Luceafarul Oradea,Sab 10:00,0.000,None
265,2.60,K Sibenik,Croácia - 2.HNL,3.10,HNK Sibenik,2.60,HNK Sibeni,Sab 09:00,0.000,None
179,2.60,TSV Landsberg,Alemanha - Oberliga - Bayern South,3.20,SV Pullach v TSV Landsberg,2.60,SV Pullach,Sab 10:00,0.000,None
278,2.87,nløse,Dinamarca - 2.ª Divisão - Grupo 1,2.75,Vanløse,2.87,Vanløs,Sab 09:00,0.000,None
545,2.45,Escócia - Feminino,Feminino - Internacional,3.50,Islândia - Feminino v Escócia - Feminino,2.45,Islândia - Feminino,Sab 11:00,0.000,None
461,3.10,ni Altindag Belediyespor,Turquia - 3 Lig - Grupo 3,2.25,Yeni Altindag Belediyespor,3.10,Yeni Altindag Belediyespo,running,0.000,None
320,2.62,zito FC,Quênia - Premier League,3.10,Wazito FC,2.60,Wazito F,Sab 09:00,0.020,None
87,2.62,Dundee,Escócia - Championship,3.30,Ayr v Dundee,2.60,Ayr,Sab 12:00,0.020,None
408,2.55,FK Poprad,Eslováquia - 2. Liga,3.40,FC Kosice v FK Poprad,2.45,FC Kosice,Sab 10:30,0.100,None


In [ ]:
'''
    INPLEMENTAR:
    * confere no google quantos jogos deram gol antes dos 10 minutos
    * tratamento do tempo para registrar exatamente o dia e hora do evento
    * verificar se cada jogo possui o mercado de minutos
    

'''

In [28]:
for g in data_games_today:
    if 'Hertha Berlim' in g['game_name']:
        print(g)

{'championship': 'Alemanha - Bundesliga I', 'time': 'Sab 11:30', 'game_name': 'Hertha Berlim v Werder Bremen', 'back_team': 'Werder Bremen', 'home_team': 'Hertha Berlim', 'back_odd': '3.10', 'home_odd': '2.25', 'draw_odd': '3.50'}


In [8]:
games = c.find_all('div',{'class':'sl-CouponParticipantWithBookCloses_NameContainer'})

NameError: name 'c' is not defined

In [214]:
game_name = [game.text for game in games]

In [215]:
markets = campeonatos[-1].find_all('div',{'class':'ufm-MarketC4OddsSwitch'})

In [217]:
markets[-1].text

'2501.005.50'

In [199]:
for c in campeonatos:
    championship = c.find('div',{'gll-MarketColumnHeader'}).text
    games = c.find_all('div',{'class':'sl-CouponParticipantWithBookCloses_NameContainer'})
    games = [game.text for game in games]
    markets = c.find_all('div',{'class':'ufm-MarketC4OddsSwitch'})
    market_odds = {'home_odd': (markets[0].text[1:]), 
                  'draw_odd': markets[1].text[1:], 
                  'back_odd':markets[2].text[1:]}
    
    print('\nCAMPEONATO: {} \n'.format(championship))
    print('JOGOS:\n  {}\n'.format(games))
    print('  HOME: {} | DRAW: {} | BACK: {} \n'.format(market_odds['home_odd'], market_odds['draw_odd'], market_odds['back_odd']))
    


CAMPEONATO: Brasil - Campeonato Acreano 

JOGOS:
  ['Sao Francisco AC1-4Andira EC']

  HOME: 51.00 | DRAW: 41.00 | BACK: 1.004 


CAMPEONATO: Brasil - Campeonato Capixaba 

JOGOS:
  ['CA Itapemirim v Estrela do Norte']

  HOME: 2.60 | DRAW: 3.20 | BACK: 2.60 


CAMPEONATO: Brasil - Campeonato Carioca 

JOGOS:
  ['Botafogo1-1Boavista SC Saquarema']

  HOME: 4.75 | DRAW: 1.50 | BACK: 5.50 


CAMPEONATO: Brasil - Campeonato Catarinense 

JOGOS:
  ['Tubarão v Avaí']

  HOME: 5.50 | DRAW: 3.25 | BACK: 1.65 


CAMPEONATO: Brasil - Campeonato Gaúcho 

JOGOS:
  ['São José PA v Ypiranga RS']

  HOME: 1.85 | DRAW: 3.50 | BACK: 3.75 


CAMPEONATO: Brasil - Campeonato Gaúcho 2 

JOGOS:
  ['São Paulo RS0-0Avenida']

  HOME: 2.40 | DRAW: 2.40 | BACK: 3.75 


CAMPEONATO: Brasil - Campeonato Mato-Grossense 

JOGOS:
  ['União Rondonópolis1-1Clube Esportivo Operário Várzea-Grandense']

  HOME: 6.00 | DRAW: 1.28 | BACK: 7.00 


CAMPEONATO: Brasil - Campeonato Mineiro 

JOGOS:
  ['Boa1-1Atlético Mineiro'

'1.083'

<div class="gll-MarketGroupContainer"></div>


CAMPEONATO: Brasil - Campeonato Acreano67:07Sao Francisco AC1-2Andira EC121.00X8.0021.083 11 

JOGOS:
  Sao Francisco AC1-2Andira EC

  HOME: 21.00 | DRAW: 8.00 | BACK: 1.083 


CAMPEONATO: Brasil - Campeonato CapixabaSeg 20:15CA Itapemirim v Estrela do Norte12.60X3.2022.60 18 

JOGOS:
  CA Itapemirim v Estrela do Norte

  HOME: 2.60 | DRAW: 3.20 | BACK: 2.60 


CAMPEONATO: Brasil - Campeonato Carioca57:04Botafogo1-1Boavista SC Saquarema13.10X2.1023.75 36 

JOGOS:
  Botafogo1-1Boavista SC Saquarema

  HOME: 3.10 | DRAW: 2.10 | BACK: 3.75 


CAMPEONATO: Brasil - Campeonato Catarinense91:11Criciúma1-3BrusqueSeg 20:00Tubarão v Avaí12.305.50X3.103.2523.001.65 229 

JOGOS:
  Criciúma1-3Brusque

  HOME: 2.305.50 | DRAW: 3.103.25 | BACK: 3.001.65 


CAMPEONATO: Brasil - Campeonato GaúchoSeg 20:00São José PA v Ypiranga RS11.85X3.5023.75 29 

JOGOS:
  São José PA v Ypiranga RS

  HOME: 1.85 | DRAW: 3.50 | BACK: 3.75 


CAMPEONATO: Brasil - Campeonato Gaúcho 2Dom 20:0018:23São Paulo RS0-0Avenid

In [28]:
home_markets[6].text

'1.36'